In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import cudf
import glob
import gc
import pickle
import os

from tqdm import tqdm

In [4]:
import random 
def partition (list_in, n):
    random.shuffle(list_in)
    return [list_in[i::n] for i in range(n)]

In [8]:
import pickle

all_sessions = pickle.load(open('./data/sessions_eval.pickle', 'rb'))
sub_sessions = pickle.load(open('./data/split/sub_sessoins_10.pickle', 'rb'))

In [10]:
for igfold in range(5):
    files = glob.glob('./data_folds/fold_' + str(igfold) + '/candidates/train/*/cand.parquet')
    files += glob.glob('./data_folds/fold_' + str(igfold) + '/candidates/sub/*/cand.parquet')
    print(files)
    sessions = pickle.load(open('./data/sessions_eval.pickle', 'rb'))
    if igfold == 0:
        sessions = [sessions[0], sessions[1]]
    elif igfold == 1:
        sessions = [sessions[2], sessions[3]]
    elif igfold == 2:
        sessions = [sessions[4], sessions[5]]
    elif igfold == 3:
        sessions = [sessions[6], sessions[7]]
    elif igfold == 4:
        sessions = [sessions[8], sessions[9]]
    for file in tqdm(files):
        df = cudf.read_parquet(file)
        os.system('mkdir -p ' + '/'.join(file.replace('./data_folds/fold_' + str(igfold) + '/', './data_folds/fold_' + str(igfold) + '/split/').split('/')[:-1]))
        if '/sub/' in file:
            for i, session_chunk in enumerate(sub_sessions):
                df_tmp = df[df['session'].isin(session_chunk)]
                df_tmp.to_parquet(
                    '/'.join(file.replace('./data_folds/fold_' + str(igfold) + '/', './data_folds/fold_' + str(igfold) + '/split/').split('/')[:-1]) + '/cand_' + str(i) + '.parquet'
                )
        else:
            for i, session_chunk in enumerate(sessions):
                df_tmp = df[df['session'].isin(session_chunk)]
                df_tmp.to_parquet(
                    '/'.join(file.replace('./data_folds/fold_' + str(igfold) + '/', './data_folds/fold_' + str(igfold) + '/split/').split('/')[:-1]) + '/cand_' + str(i) + '.parquet'
                )

In [11]:
clicks_ignore = [
    'weighted_0_2_dummy', 'weighted_0_2_count', 'num_aid_ca',
       'num_aid_ca_fs', 'c2o_num_15', 'weighted_0_2_wgt_sum_2',
       'num_aid_fs', 'b2b_rank_min_15', 'caco_wgt_ca_15', 'b2b_wgt_ca_15',
       'clicks_wgt_or_20', 'num_u_aid_cl_fs', 'caco_wgt_15',
       'b2b_wgt_2_15', 'carts', 'caco_wgt_2_15', 'c2o_wgt_ca_15',
       'clicks_wgt_ca_20', 'orders', 'c2o_wgt_or_15',
       'weighted_0_2_wgt_prob_2', 'num_sessions', 'caco_b2b_rank_min_15',
       'caco_b2b_wgt_15', 'caco_b2b_wgt_2_15', 'caco_b2b_wgt_cl_15',
       'caco_b2b_wgt_ca_15', 'caco_b2b_wgt_or_15', 'num_aid_co',
       'caco_b2b_num_15', 'num_u_aid_ca_fs', 'num_aid_co_fs',
       'num_u_aid_co', 'num_u_aid_co_fs', 'b2b_wgt_or_15', 'b2b_wgt_15',
       'weighted_0_2_wgt_sum_2_2', 'caco_wgt_or_15', 'b2b_num_15',
       'c2o_wgt_2_15', 'caco_num_15', 'clicks_num_20', 'b2b_wgt_cl_15',
       'weighted_0_2_count_2', 'c2o_wgt_cl_15', 'weighted_0_2_wgt_prob',
       'num_u_aid_ca', 'num_aid_cl_fs', 'caco_wgt_cl_15'
]

cart_ignore = ['num_sessions', 'caco_wgt_15', 'b2b_rank_min_15', 'b2b_wgt_ca_15',
       'clicks_wgt_2_20', 'num_aid_co', 'weighted_0_2_dummy',
       'caco_num_15', 'ts_diff', 'num_aid_fs', 'caco_wgt_cl_15',
       'c2o_wgt_cl_15', 'num_u_aid_co_fs', 'num_aid_co_fs',
       'caco_wgt_or_15', 'c2o_wgt_or_15', 'caco_wgt_ca_15',
       'num_u_aid_co', 'baseline', 'caco_b2b_wgt_ca_15',
       'caco_b2b_wgt_cl_15', 'caco_b2b_wgt_2_15', 'caco_b2b_wgt_15',
       'caco_b2b_rank_min_15', 'caco_b2b_num_15', 'orders',
       'caco_b2b_wgt_or_15', 'baseline2', 'b2b_wgt_or_15',
       'clicks_wgt_or_20', 'c2o_wgt_ca_15', 'clicks_wgt_ca_20',
       'weighted_0_2_count', 'num_u_aid_cl_fs']

order_ignore = [
    'clicks_num_20', 'num_u_aid', 'num_aid_ca', 'caco_b2b_wgt_or_15',
    'baseline', 'caco_b2b_wgt_cl_15', 'caco_rank_min_15',
    'caco_wgt_or_15', 'caco_b2b_wgt_ca_15', 'orders', 'baseline2',
    'caco_b2b_rank_min_15', 'caco_b2b_wgt_15', 'caco_b2b_wgt_2_15',
    'caco_b2b_num_15', 'b2b_wgt_ca_15', 'num_sessions',
       'caco_wgt_ca_15', 'num_aid_cl_fs', 'weighted_count_30',
       'num_u_aid_co_fs', 'caco_wgt_15', 'weighted_0_2_count',
       'num_aid_co_fs']

total_ignore = [x for x in order_ignore if x in cart_ignore and x in clicks_ignore]
len(total_ignore)

no_cands = 100

In [12]:
mainprefix = 'chris_'


total_ignore = total_ignore + [zz for z in [[
    mainprefix + '_' + str(i) + '_' + x
    for x in ['_l_dummy', '_l_wgt_2', '_l_dummy', '_d_wgt_1', '_t_wgt_2', '_t_dummy']
] for i in range(15)] for zz in z ]

In [13]:
list_sort_columns = [
    'c2o_wgt_15', 
    'clicks_wgt_20', 
    'b2b_wgt_15', 
    'caco_wgt_15', 
    'pv_wgt_1', 
    'weighted_wgt_30', 
    'weighted_1_2_wgt_sum_2',
    't4r_score',
    'caco_b2b_wgt_15',
    'pv_2_wgt_1',
    'caco_df__wgt_15',
    'chris__0__wgt_1',
    'chris__1__wgt_1',
    'chris__2__wgt_1',
    'chris__3__wgt_1',
    'chris__4__wgt_1',
    'chris__5__wgt_1',
    'chris__6__wgt_1',
    'chris__7__wgt_1',
    'chris__8__wgt_1',
    'chris__9__wgt_1'
]

In [14]:
ignore_files = [
    'chris__10__15', 'chris__9__15', 'chris__8__15', 'chris__7__15', 'chris__6__15'
]
ignore_files = [
    'chris__0__15', 'chris__1__15', 'chris__2__15', 'chris__3__15', 'chris__4__15', 'chris__5__15'
]
ignore_files = ['xxxnonexxx'] + [
    'weighted_last_item_30', 'weighted_last_hour_30', 'weighted_last_click_item_30', 'weighted_last_caco_item_30',
    'clicks_last_item_20', 'clicks_last_hour_20', 'clicks_last_click_item_20', 'clicks_last_caco_item_20',
    'c2o_last_hour_clicks_15', 'c2o_last_click_item_15', 'weighted_last_item_30', 'weighted_last_hour_30',
    'weighted_last_click_item_30', 'weighted_last_caco_item_30', 'c2o_last_click_item_15', 'c2o_last_hour_clicks_15',
    'clicks_last_caco_item_20', 'clicks_last_click_item_20', 'clicks_last_hour_20', 'clicks_last_item_20', 'weighted_lw_30'
]

In [17]:
val_cols = [
    'p3w_clicks',
    'p0w_clicks',
    'weighted_wgt_max_30',
    'p3w_carts',
    't4r_score',
    'p3w_orders',
    'chris__6__wgt',
    'weighted_wgt_30',
    'chris__4__wgt_1',
    'num_aid_fs',
    'chris__0__wgt',
    'chris__5__wgt_1',
    'c2o_wgt_2_15',
    'weighted_count_30',
    'num_u_aid_cl',
    'b2b_wgt_2_15'
]

In [93]:
def get_cands(files, get_train=True):
    cand_list = []
    for file in files:
#        if any([True for x in [
#            '/c2o_15/', '/clicks_20/', '/b2b_15/', '/caco_15/', 
#            '/pv/', '/weighted_30/', '/caco_b2b_15/', '/t4r/', '/weighted_1_2_20/', 
#            '/chris__0__15/', #'/chris__5__15/', '/chris__8__15/',
#        ] if x in file]):
        if True:
            print(file)
            df = cudf.read_parquet(file)
            if get_train:
                print('Filter')
                df = df[df['session'].isin(labels_filter)]
            sort_col = [x for x in df.columns if x in list_sort_columns]
            assert len(sort_col) == 1
            df['dummy'] = 1
            df = df.sort_values(sort_col, ascending=False)
            df['srank'] = df.groupby(['session']).dummy.cumsum()
            df = df[df['srank']<=no_cands]
            cand_list.append(df[['session', 'cand', 'dummy']])

    df_cand = cudf.concat(cand_list)
    df_cand = df_cand[['session', 'cand', 'dummy']].groupby(['session', 'cand']).sum().reset_index()
    print(df_cand['dummy'].value_counts())
    df_cand = df_cand[df_cand['dummy']>1]
    print(df_cand.shape)
    del cand_list, df
    gc.collect()
    df_cand = df_cand.drop_duplicates(['session', 'cand'])
    print('Merge')
    for file in files:
        if not any([True for x in ignore_files if x in file]):
            print(file)
            df_tmp = cudf.read_parquet(file)
            df_cand = df_cand.merge(
                df_tmp[[x for x in df_tmp.columns if x in ['session', 'cand'] or x not in total_ignore]],
                #df_tmp,
                how='left',
                on=['session', 'cand']
            )
            cols = [x for x in df_tmp.columns if x not in ['session', 'cand'] and x not in total_ignore]
            #cols = [x for x in df_tmp.columns if x not in ['session', 'cand']]
            del df_tmp
            df_cand[cols] = df_cand[cols].fillna(0)
            gc.collect()
    return(df_cand)

In [19]:
labels = cudf.read_parquet('./data/xgb_train_y.parquet')
labels.columns = ['session', 'cand', 'type']
labels['target'] = 1

In [20]:
labels_filter = labels[labels['type'].isin(['carts', 'orders'])]['session'].drop_duplicates()

In [22]:
df_day_map = cudf.DataFrame({
    'day': [21, 22, 23, 24, 25, 26, 27, 28],
    'day_': [0, 0, 1, 2, 3, 4, 5, 6]
})

df_session_ts = cudf.read_parquet('./data/xgb_train_x.parquet')
df_session_ts = df_session_ts.groupby(['session']).ts.max().reset_index()
df_session_ts['day'] = cudf.to_datetime(df_session_ts['ts']/1000, unit='s').dt.day
df_session_ts = df_session_ts.merge(
    df_day_map,
    how='left',
    on=['day']
)
df_session_ts.drop(['ts', 'day'], axis=1, inplace=True)
df_cand_ts = cudf.read_parquet('./data/' + 'train' + '_cand_day_meta_comb.parquet')
df_cand_ts = df_cand_ts.rename(columns={'aid': 'cand'})

In [ ]:
for igfold in range(5):
    session_meta = cudf.read_parquet('./data_folds/fold_' + str(igfold) + '/train_sessions.parquet')
    cand_meta = cudf.read_parquet('./data_folds/fold_' + str(igfold) + '/' + 'train' + '_cand_meta_comb.parquet')
    for col in cand_meta.columns:
        cand_meta[col] = cand_meta[col].astype('int32')
    cand_meta = cand_meta.rename(columns={'aid': 'cand'})
    os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/split/chunks/carts')
    os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/split/chunks/orders')
    
    for i in range(2):
        files = glob.glob('./data_folds/fold_' + str(igfold) + '/split/candidates/train/*/cand_' + str(i) + '.parquet') 
        df = get_cands(files)
        df = df.merge(
            cand_meta,
            how='left',
            on=['cand']
        )
        df[[x for x in cand_meta.columns if x != 'cand']] = df[[x for x in cand_meta.columns if x != 'cand']].fillna(0)
        df = df.merge(
            session_meta,
            how='left'
        )
        dtypes = list(df.dtypes.reset_index().values)
        for dtype in dtypes:
            if dtype[1].name == 'float64':
                df[dtype[0]] = df[dtype[0]].astype('float32')
        df = df.merge(
            df_session_ts,
            how='left',
            on=['session']
        )
        df = df.merge(
            df_cand_ts,
            how='left',
            on=['cand', 'day_']
        )
        df[['sday_clicks', 'sday_carts', 'sday_orders']] = df[['sday_clicks', 'sday_carts', 'sday_orders']].fillna(0)
        print(df.shape)
        for ty in ['carts']:
            tmp_labels = labels[labels['type']==ty][['session', 'cand', 'target']]
            if ty != 'clicks':
                df_tmp = df[df['session'].isin(tmp_labels['session'].drop_duplicates())].reset_index(drop=True)
            df_tmp = df_tmp.merge(
                labels[labels['type']==ty][['session', 'cand', 'target']],
                how='left',
                on=['session', 'cand']
            )
            df_tmp['target'] = df_tmp['target'].fillna(0)
            gc.collect()
            df_tmp.to_parquet('./data_folds/fold_' + str(igfold) + '/split/chunks/' + str(ty) + '/chunk_' + str(i) + '.parquet')
            df_tmp.drop(['target'], inplace=True, axis=1)
            del df_tmp
            gc.collect()

        del df
        gc.collect()

In [23]:
df_day_map = cudf.DataFrame({
    'day': [28, 29, 30, 31, 1, 2, 3, 4],
    'day_': [0, 0, 1, 2, 3, 4, 5, 6]
})

df_session_ts = cudf.read_parquet('../../data/test.parquet')
df_session_ts = df_session_ts.groupby(['session']).ts.max().reset_index()
df_session_ts['day'] = cudf.to_datetime(df_session_ts['ts']/1000, unit='s').dt.day
df_session_ts = df_session_ts.merge(
    df_day_map,
    how='left',
    on=['day']
)
df_session_ts.drop(['ts', 'day'], axis=1, inplace=True)
df_cand_ts = cudf.read_parquet('./data/' + 'sub' + '_cand_day_meta_comb.parquet')
df_cand_ts = df_cand_ts.rename(columns={'aid': 'cand'})

In [24]:
for igfold in range(5):
    session_meta = cudf.read_parquet('./data_folds/fold_' + str(igfold) + '/sub_sessions.parquet')
    cand_meta = cudf.read_parquet('./data_folds/fold_' + str(igfold) + '/' + 'sub' + '_cand_meta_comb.parquet')
    for col in cand_meta.columns:
        cand_meta[col] = cand_meta[col].astype('int32')
    cand_meta = cand_meta.rename(columns={'aid': 'cand'})
    os.system('rm -rf ./data_folds/fold_' + str(igfold) + '/split/sub/')
    os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/split/sub/')
    
    for i in range(len(sub_sessions)):
        files = glob.glob('./data_folds/fold_' + str(igfold) + '/split/candidates/sub/*/cand_' + str(i) + '.parquet')
        df = get_cands(files, get_train=False)
        print(df.shape)
        df = df.merge(
            cand_meta,
            how='left',
            on=['cand']
        )
        df[[x for x in cand_meta.columns if x != 'cand']] = df[[x for x in cand_meta.columns if x != 'cand']].fillna(0)
        df = df.merge(
            session_meta,
            how='left'
        )
        df = df.merge(
            df_session_ts,
            how='left',
            on=['session']
        )
        df = df.merge(
            df_cand_ts,
            how='left',
            on=['cand', 'day_']
        )
        df[['sday_clicks', 'sday_carts', 'sday_orders']] = df[['sday_clicks', 'sday_carts', 'sday_orders']].fillna(0)
        print(df.shape)
        df.to_parquet('./data_folds/fold_' + str(igfold) + '/split/sub/chunk_' + str(i) + '.parquet')
        del df
        gc.collect()

./data_folds/fold_0/split/candidates/sub/c2o_15/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/clicks_20/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/pv_2/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/chris__2__15/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/b2b_15/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/chris__3__15/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/chris__0__15/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/chris__8__15/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/caco_15/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/pv/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/chris__5__15/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/weighted_30/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/chris__4__15/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/weighted_1_2_20/cand_0.parquet
./data_folds/fold_0/split/candidates/sub/t4r/cand_0.parquet
./data_fol

./data_folds/fold_0/split/candidates/sub/pv/cand_2.parquet
./data_folds/fold_0/split/candidates/sub/chris__5__15/cand_2.parquet
./data_folds/fold_0/split/candidates/sub/weighted_30/cand_2.parquet
./data_folds/fold_0/split/candidates/sub/chris__4__15/cand_2.parquet
./data_folds/fold_0/split/candidates/sub/weighted_1_2_20/cand_2.parquet
./data_folds/fold_0/split/candidates/sub/t4r/cand_2.parquet
./data_folds/fold_0/split/candidates/sub/chris__9__15/cand_2.parquet
./data_folds/fold_0/split/candidates/sub/caco_b2b_15/cand_2.parquet
./data_folds/fold_0/split/candidates/sub/chris__7__15/cand_2.parquet
./data_folds/fold_0/split/candidates/sub/chris__6__15/cand_2.parquet
./data_folds/fold_0/split/candidates/sub/chris__1__15/cand_2.parquet
(9082051, 129)
(9082051, 163)
./data_folds/fold_0/split/candidates/sub/c2o_15/cand_3.parquet
./data_folds/fold_0/split/candidates/sub/clicks_20/cand_3.parquet
./data_folds/fold_0/split/candidates/sub/pv_2/cand_3.parquet
./data_folds/fold_0/split/candidates/su

Merge
./data_folds/fold_0/split/candidates/sub/c2o_15/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/clicks_20/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/pv_2/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/chris__2__15/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/b2b_15/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/chris__3__15/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/chris__0__15/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/chris__8__15/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/caco_15/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/pv/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/chris__5__15/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/weighted_30/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/chris__4__15/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/weighted_1_2_20/cand_5.parquet
./data_folds/fold_0/split/candidates/sub/t4r/cand_5.parquet
./da

./data_folds/fold_0/split/candidates/sub/weighted_1_2_20/cand_8.parquet
./data_folds/fold_0/split/candidates/sub/t4r/cand_8.parquet
./data_folds/fold_0/split/candidates/sub/chris__9__15/cand_8.parquet
./data_folds/fold_0/split/candidates/sub/caco_b2b_15/cand_8.parquet
./data_folds/fold_0/split/candidates/sub/chris__7__15/cand_8.parquet
./data_folds/fold_0/split/candidates/sub/chris__6__15/cand_8.parquet
./data_folds/fold_0/split/candidates/sub/chris__1__15/cand_8.parquet
1     9826065
2     2788979
3     1483716
4     1009552
5      741221
6      599279
7      447373
8      368025
9      314771
10     272875
15     234478
11     207770
14     201069
12     186894
13     172395
16      28334
17      22632
18       5881
Name: dummy, dtype: int32
(9085244, 3)
Merge
./data_folds/fold_0/split/candidates/sub/c2o_15/cand_8.parquet
./data_folds/fold_0/split/candidates/sub/clicks_20/cand_8.parquet
./data_folds/fold_0/split/candidates/sub/pv_2/cand_8.parquet
./data_folds/fold_0/split/candidates/

./data_folds/fold_0/split/candidates/sub/pv_2/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/chris__2__15/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/b2b_15/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/chris__3__15/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/chris__0__15/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/chris__8__15/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/caco_15/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/pv/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/chris__5__15/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/weighted_30/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/chris__4__15/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/weighted_1_2_20/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/t4r/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/chris__9__15/cand_11.parquet
./data_folds/fold_0/split/candidates/sub/caco_b2b_15/can

...

./data_folds/fold_4/split/candidates/sub/chris__0__15/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/chris__8__15/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/caco_15/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/pv/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/chris__5__15/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/weighted_30/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/chris__4__15/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/weighted_1_2_20/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/t4r/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/chris__9__15/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/caco_b2b_15/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/chris__7__15/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/chris__6__15/cand_12.parquet
./data_folds/fold_4/split/candidates/sub/chris__1__15/cand_12.parquet
1     9836223
2     2793015
3     1487810
4

./data_folds/fold_4/split/candidates/sub/weighted_1_2_20/cand_14.parquet
./data_folds/fold_4/split/candidates/sub/t4r/cand_14.parquet
./data_folds/fold_4/split/candidates/sub/chris__9__15/cand_14.parquet
./data_folds/fold_4/split/candidates/sub/caco_b2b_15/cand_14.parquet
./data_folds/fold_4/split/candidates/sub/chris__7__15/cand_14.parquet
./data_folds/fold_4/split/candidates/sub/chris__6__15/cand_14.parquet
./data_folds/fold_4/split/candidates/sub/chris__1__15/cand_14.parquet
(9073326, 129)
(9073326, 163)
./data_folds/fold_4/split/candidates/sub/c2o_15/cand_15.parquet
./data_folds/fold_4/split/candidates/sub/clicks_20/cand_15.parquet
./data_folds/fold_4/split/candidates/sub/pv_2/cand_15.parquet
./data_folds/fold_4/split/candidates/sub/chris__2__15/cand_15.parquet
./data_folds/fold_4/split/candidates/sub/b2b_15/cand_15.parquet
./data_folds/fold_4/split/candidates/sub/chris__3__15/cand_15.parquet
./data_folds/fold_4/split/candidates/sub/chris__0__15/cand_15.parquet
./data_folds/fold_4/

Merge
./data_folds/fold_4/split/candidates/sub/c2o_15/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/clicks_20/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/pv_2/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/chris__2__15/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/b2b_15/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/chris__3__15/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/chris__0__15/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/chris__8__15/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/caco_15/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/pv/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/chris__5__15/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/weighted_30/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/chris__4__15/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/weighted_1_2_20/cand_17.parquet
./data_folds/fold_4/split/candidates/sub/t4r/cand_

./data_folds/fold_4/split/candidates/sub/chris__4__15/cand_20.parquet
./data_folds/fold_4/split/candidates/sub/weighted_1_2_20/cand_20.parquet
./data_folds/fold_4/split/candidates/sub/t4r/cand_20.parquet
./data_folds/fold_4/split/candidates/sub/chris__9__15/cand_20.parquet
./data_folds/fold_4/split/candidates/sub/caco_b2b_15/cand_20.parquet
./data_folds/fold_4/split/candidates/sub/chris__7__15/cand_20.parquet
./data_folds/fold_4/split/candidates/sub/chris__6__15/cand_20.parquet
./data_folds/fold_4/split/candidates/sub/chris__1__15/cand_20.parquet
1     9860006
2     2801087
3     1488742
4     1007587
5      742253
6      599640
7      447056
8      367328
9      313837
10     272379
15     232159
11     207135
14     199989
12     187452
13     172712
16      27894
17      22176
18       5608
Name: dummy, dtype: int32
(9095034, 3)
Merge
./data_folds/fold_4/split/candidates/sub/c2o_15/cand_20.parquet
./data_folds/fold_4/split/candidates/sub/clicks_20/cand_20.parquet
./data_folds/fold_4

./data_folds/fold_4/split/candidates/sub/chris__1__15/cand_22.parquet
(9087131, 129)
(9087131, 163)
./data_folds/fold_4/split/candidates/sub/c2o_15/cand_23.parquet
./data_folds/fold_4/split/candidates/sub/clicks_20/cand_23.parquet
./data_folds/fold_4/split/candidates/sub/pv_2/cand_23.parquet
./data_folds/fold_4/split/candidates/sub/chris__2__15/cand_23.parquet
./data_folds/fold_4/split/candidates/sub/b2b_15/cand_23.parquet
./data_folds/fold_4/split/candidates/sub/chris__3__15/cand_23.parquet
./data_folds/fold_4/split/candidates/sub/chris__0__15/cand_23.parquet
./data_folds/fold_4/split/candidates/sub/chris__8__15/cand_23.parquet
./data_folds/fold_4/split/candidates/sub/caco_15/cand_23.parquet
./data_folds/fold_4/split/candidates/sub/pv/cand_23.parquet
./data_folds/fold_4/split/candidates/sub/chris__5__15/cand_23.parquet
./data_folds/fold_4/split/candidates/sub/weighted_30/cand_23.parquet
./data_folds/fold_4/split/candidates/sub/chris__4__15/cand_23.parquet
./data_folds/fold_4/split/can